In [2]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import numpy as np

## Parameters Setup

In [3]:
# parameters
MAX_LENGTH = 25
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

SOS_token = 1
EOS_token = 2

hidden_size = 512

batch_size = 1

cpu


## Data Setup

In [4]:
# set data path
data_dir = os.path.join('datasets', 'nmt_data_vi')
train_source = 'train.vi'
train_target = 'train.en'
train_source_dir = os.path.join(data_dir, train_source)
train_target_dir = os.path.join(data_dir, train_target)
vocab_source = 'vocab.vi'
vocab_target = 'vocab.en'
vocab_source_dir = os.path.join(data_dir, vocab_source)
vocab_target_dir = os.path.join(data_dir, vocab_target)

In [5]:
# load training sets
with open(train_source_dir) as f_source:
    sentences_source = f_source.readlines()
with open(train_target_dir) as f_target:
    sentences_target = f_target.readlines()

# check the total number of sentencs in training sets    
print("Total number of sentences in source training set: {}".format(len(sentences_source)))
print("Total number of sentences in target training set: {}".format(len(sentences_target)))

Total number of sentences in source training set: 133317
Total number of sentences in target training set: 133317


In [6]:
# Truncate sentences by maximum length
sentences_source = list(map(lambda src:src.split()[:MAX_LENGTH], sentences_source))
sentences_target = list(map(lambda src:src.split()[:MAX_LENGTH], sentences_target))

# check the longest sentence after sentence truncation
max = 0
for s in sentences_source:
    if len(s) > max:
        max = len(s)
        max_s = s
print("Number of words in the longest sentence in sentences_source: {}".format(max))
print("The longest sentence: \n{}".format(max_s))

Number of words in the longest sentence in sentences_source: 25
The longest sentence: 
['Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng']


In [27]:
# load vocabularies

# build index2word
with open(vocab_source_dir) as f_vocab_source:
    #index2word_source = f_vocab_source.readlines()
    index2word_source = [line.rstrip() for line in f_vocab_source]
with open(vocab_target_dir) as f_vocab_target:
    #index2word_target = f_vocab_target.readlines()
    index2word_target = [line.rstrip() for line in f_vocab_target]

index2word_target = np.array(index2word_target)
index2word_source = np.array(index2word_source)

# build word2index
word2index_source = {}
for idx, word in enumerate(index2word_source):
    word2index_source[word] = idx
word2index_target = {}
for idx, word in enumerate(index2word_target):
    word2index_target[word] = idx
    
# check vocabularies size    
source_vocab_size = len(index2word_source)
target_vocab_size = len(index2word_target)
print("Total nummber of words in source vocabulary: {}".format(len(index2word_source)))
print("Total nummber of words in target vocabulary: {}".format(len(index2word_target)))    

Total nummber of words in source vocabulary: 7709
Total nummber of words in target vocabulary: 17191


In [8]:
# helper funtions to convert sentence in natural language to list of word indexes
def sen2idx(sentence, word2index):
    return [word2index.get(word, 0) for word in sentence] # assume that 0 is for <unk>

def sen2tensor(sentence, word2index):
    idxes = sen2idx(sentence, word2index)
    idxes.append(EOS_token)
    return torch.tensor(idxes, dtype=torch.long, device=device)

## Token to be ignored

In [9]:
PAD_token = target_vocab_size # padding value

## Batch Generator

In [9]:
# batch generator
def sentences2tensor(sentences, word2index):
    sentences_tensor = [sen2tensor(s, word2index) for s in sentences]
    sentences_tensor.sort(key=len, reverse=True)
    output = pad_sequence(sentences_tensor, batch_first=True)
    return output

def batch_generator(batch_size, sentences_source, sentences_target, word2index_source, word2index_target):
    #output: two PackedSequence object, two indexes to reorder sentences in a batch
    
    # generate id in one batch
    total = len(sentences_source)
    sample_id = np.random.choice(total, batch_size, replace=False)
    
    #generate a source batch
    sentences_source_tensor = [sen2tensor(sentences_source[id], word2index_source) for id in sample_id]
    
    len_array_source = [len(st) for st in sentences_source_tensor]
    reorder_idx_source = np.argsort(len_array_source, kind='mergesort')
    reorder_idx_source = np.argsort(np.flip(reorder_idx_source, 0)) #index to restore unsorted order
    
    sentences_source_tensor.sort(key=len)
    sentences_source_tensor.reverse()
    sentences_source_packed = pack_sequence(sentences_source_tensor)
    
    #generate a target batch
    sentences_target_tensor = [sen2tensor(sentences_target[id], word2index_target) for id in sample_id]
    
    len_array_target = [len(st) for st in sentences_target_tensor]
    reorder_idx_target = np.argsort(len_array_target, kind='mergesort')
    reorder_idx_target = np.argsort(np.flip(reorder_idx_target, 0)) #index to restore unsorted order
    
    sentences_target_tensor.sort(key=len)
    sentences_target_tensor.reverse()
    sentences_target_packed = pack_sequence(sentences_target_tensor)
    
    return (sentences_source_packed, reorder_idx_source), (sentences_target_packed, reorder_idx_target)

In [10]:
# test batch_generator
a = ['I','am','a','boy']
b = ['a']
c = ['the','goat']

sentences_test = [a,b,c]
sentences_test2 = [c,b,a]

(output_test, reorder_idx_test), (output_test2, reorder_idx_test2) = batch_generator(2, sentences_test, sentences_test2, word2index_target, word2index_target)
print(output_test)
print(reorder_idx_test)
print(pad_packed_sequence(output_test, batch_first=True))

del a, b, c, sentences_test, sentences_test2, output_test, reorder_idx_test, output_test2, reorder_idx_test2

PackedSequence(data=tensor([   20,     8,  1352,     2,     2]), batch_sizes=tensor([ 2,  2,  1]))
[0 1]
(tensor([[   20,  1352,     2],
        [    8,     2,     0]]), tensor([ 3,  2]))


## Helper function to resume order of sentences in a batch

Order recovery is necessary because sentences have to be sorted in descending order of sentence length to be packed as a PackedSequence object. PackedSequence object helps to deal with inputs with variable length in NMT setting. LSTM, RNN can accept PackedSequence objects.

In [10]:
def resume_order(input, idx):
    # input 
    #   input: Tensor: (batch_size, seq_length)
    #   idx: Tensor or ndarray: (batch_size)
    # output
    #   out: Tensor with reordered sentences in batch: (batch_size, seq_length) 
    
    if isinstance(idx, (np.ndarray)):
        idx = torch.from_numpy(idx)
        if device == torch.device("cuda"):
            idx = idx.cuda()
    out = torch.index_select(input, 0, idx)
    return out

In [11]:
# test resume_order
input_test = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],device=device)
idx_test = np.array([2,1,0])
print(resume_order(input_test, idx_test))
input_test = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],device=device)
idx_test = torch.tensor([2,1,0], device=device)
print(resume_order(input_test, idx_test))
print(idx_test.device)
del input_test, idx_test

tensor([[ 7,  8,  9],
        [ 4,  5,  6],
        [ 1,  2,  3]])
tensor([[ 7,  8,  9],
        [ 4,  5,  6],
        [ 1,  2,  3]])
cpu


## Define Encoder and Decoder classes

In [12]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers=1, num_directions=1, dropout=0):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.num_directions = num_directions
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
    def forward(self, input_tuple, prev_h, prev_c):
        # input
        # input size: (batch_size, seq_length)
        # prev_h size: (num_layers*num_directions, batch_size, hidden_size)
        # prec_c size: (num_layers*num_directions, batch_size, hidden_size)
        # output
        # h_n size: (num_layers*num_directions, batch_size, hidden_size)
        # c_n size: (num_layers*num_directions, batch_size, hidden_size)
        (sentences_packed, reorder_idx) = input_tuple
        sentences_tensor, sentences_length = pad_packed_sequence(sentences_packed, batch_first=True, padding_value=0)
        #sentences_tensor: (batch_size, seq_length)
        input_embedded = self.embedding(sentences_tensor) # (batch_size, seq_length, hidden_size)
        input_embedded_packed = pack_padded_sequence(input_embedded, sentences_length, batch_first=True)
        _, (h_n, c_n) = self.lstm(input_embedded_packed, (prev_h, prev_c))
        return h_n, c_n
    def initHidden(self):
        return torch.zeros(self.num_layers*self.num_directions, self.batch_size, self.hidden_size, device=device)

In [13]:
# num_layers and num_directions for encoder must be 0 when this decoder is used
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTMCell(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, prev_h, prev_c):
        input_embedded = self.embedding(input)
        h, c = self.lstm(input_embedded, (prev_h, prev_c))
        output =self.softmax(self.out(h))
        return output, h, c
    def initHidden(self):
        return torch.zeros(self.batch_size, self.hidden_size, device=device)

In [14]:
# This DecoderLSTM can't do teacher forcing in training

# class DecoderLSTM(nn.Module):
#     def __init__(self, hidden_size, output_size, batch_size, num_layers=1, num_directions=1, dropout=0):
#         super(DecoderLSTM, self).__init__()
#         self.hidden_size = hidden_size
#         self.batch_size = batch_size
#         self.num_layers = num_layers
#         self.num_directions = num_directions
#         self.dropout = dropout
        
#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
#         self.out = nn.Linear(hidden_size*num_directions, output_size)
#         self.softmax = nn.LogSoftmax(dim=2)
#     def forward(self, input, prev_h, prev_c):
#         # input 
#         # input size: (batch_size, seq_length)
#         # prev_h size: (num_layers*num_directions, batch_size, hidden_size)
#         # prec_c size: (num_layers*num_directions, batch_size, hidden_size)
#         # output
#         # h_n size: (num_layers*num_directions, batch_size, hidden_size)
#         # c_n size: (num_layers*num_directions, batch_size, hidden_size)
#         # output size: (batch_size, seq_length, output_size)
#         input_embedded = self.embedding(input)
#         output, (h_n, c_n) = self.lstm(input_embedded, (prev_h, prev_c))
#         output =self.softmax(self.out(h))
#         return output, h_n, c_n
#     def initHidden(self, batch_size):
#         return torch.zeros(self.num_layers*self.num_directions, self.batch_size, self.hidden_size, device=device)

## Training

In [46]:
def train(source_tuple, target_tuple, encoder, decoder, encoder_optimizer, decoder_optimizer,batch_size, max_length=MAX_LENGTH):
    
    encoder_hidden_h = encoder.initHidden()
    encoder_hidden_c = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0
    
    criterion = nn.NLLLoss(ignore_index=PAD_token, size_average=False)
    
    (sentences_source_packed, reorder_idx_source) = source_tuple
    (sentences_target_packed, reorder_idx_target) = target_tuple
    sentences_target_tensor, sentences_target_length = pad_packed_sequence(sentences_target_packed, batch_first=True, padding_value=PAD_token)
    sentences_target_tensor = resume_order(sentences_target_tensor, reorder_idx_target)
    

    target_length = sentences_target_tensor.size(1)
    #print("target length: {}".format(target_length)) # to be removed
    #print("target sentence: {}".format(sentences_target_tensor)) # to be removed
    
    # encoder_hidden_h size: (num_layers*num_directions, batch_size, hidden_size)
    # encoder_hidden_c size: (num_layers*num_directions, batch_size, hidden_size)
    encoder_hidden_h, encoder_hidden_c = encoder(source_tuple, encoder_hidden_h, encoder_hidden_c)
    
    
    decoder_input = torch.full((batch_size,), SOS_token, dtype=torch.long, device=device)
    decoder_hidden_c = resume_order(encoder_hidden_c[0], reorder_idx_source)
    decoder_hidden_h = resume_order(encoder_hidden_h[0], reorder_idx_source)
    
    to_print = []
    
    for di in range(target_length):
        decoder_output, decoder_hidden_h, decoder_hidden_c = decoder(decoder_input, decoder_hidden_h, decoder_hidden_c)
        loss += criterion(decoder_output, sentences_target_tensor[:,di])
        decoder_input = sentences_target_tensor[:,di]
        
        #to be removed
        decoder_output_np = np.argmax(decoder_output.detach().cpu().numpy(), 1)
        if di == 0: 
            to_print = index2word_target[decoder_output_np]
        else:
            to_print = np.column_stack((to_print, index2word_target[decoder_output_np]))
        #####
    
    denominator = torch.sum(sentences_target_length).float()
    if device == torch.device("cuda"):
        denominator = denominator.cuda()
    loss = loss / denominator
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item(), to_print

In [45]:
# define a batch_generator for overfitting test only
# this batch_generator will generate a batch containing selected sentences only

def batch_generator(batch_size, sentences_source, sentences_target, word2index_source, word2index_target):
    #output: two PackedSequence object, two indexes to reorder sentences in a batch
    
    # generate id in one batch
    total = len(sentences_source)
#     sample_id = np.random.choice(total, batch_size, replace=False)
    sample_id = np.array([20, 26, 28, 44, 49, 52, 56, 57, 64, 66])
    
    #generate a source batch
    sentences_source_tensor = [sen2tensor(sentences_source[id], word2index_source) for id in sample_id]
    
    len_array_source = [len(st) for st in sentences_source_tensor]
    reorder_idx_source = np.argsort(len_array_source, kind='mergesort')
    reorder_idx_source = np.argsort(np.flip(reorder_idx_source, 0)) #index to restore unsorted order
    
    sentences_source_tensor.sort(key=len)
    sentences_source_tensor.reverse()
    sentences_source_packed = pack_sequence(sentences_source_tensor)
    
    #generate a target batch
    sentences_target_tensor = [sen2tensor(sentences_target[id], word2index_target) for id in sample_id]
    
    len_array_target = [len(st) for st in sentences_target_tensor]
    reorder_idx_target = np.argsort(len_array_target, kind='mergesort')
    reorder_idx_target = np.argsort(np.flip(reorder_idx_target, 0)) #index to restore unsorted order
    
    sentences_target_tensor.sort(key=len)
    sentences_target_tensor.reverse()
    sentences_target_packed = pack_sequence(sentences_target_tensor)
    
    return (sentences_source_packed, reorder_idx_source), (sentences_target_packed, reorder_idx_target)

In [ ]:
# no need to run this cell
# print out target sentences and select some to be put into the small dataset
for idx, s in enumerate(sentences_target):
    print("idx:{}, sentences:{}".format(idx,s))

## Small dataset for overfitting test

20: ['We', 'blow', 'it', 'up', 'and', 'look', 'at', 'the', 'pieces', '.'] length 11

26: ['And', 'it', 'takes', 'weeks', 'to', 'perform', 'our', 'integrations', '.'] length 10

28: ['We', 'also', 'fly', 'all', 'over', 'the', 'world', 'looking', 'for', 'this', 'thing', '.'] length 13

44: ['We', 'do', 'all', 'of', 'this', 'to', 'understand', 'the', 'chemistry', 'of', 'one', 'molecule', '.'] length 14

49: ['So', 'you', 'can', 'imagine', 'the', 'scale', 'of', 'the', 'effort', '.'] length 11

52: ['Thank', 'you', 'very', 'much', '.'] length 6

56: ['You', 'can', 'mimic', 'what', 'you', 'can', 'see', '.'] length 9

57: ['You', 'can', 'program', 'the', 'hundreds', 'of', 'muscles', 'in', 'your', 'arm', '.'] length 12

64: ['It', 'was', 'terribly', 'dangerous', '.'] length 6

66: ['But', 'now', ',', 'we', 'have', 'a', 'real', 'technology', 'to', 'do', 'this', '.'] length 13

    index to reorder [] 

In [47]:
batch_size = 10 # to be removed
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.1):
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), learning_rate)
    
    for iter in range(1, n_iters+1):
        source_tuple, target_tuple = batch_generator(batch_size, sentences_source, sentences_target, word2index_source, word2index_target)
        loss, to_print = train(source_tuple, target_tuple, encoder, decoder, encoder_optimizer, decoder_optimizer, batch_size)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter%print_every ==0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, print_loss_avg))
            print(to_print)

In [48]:
encoder1 = EncoderLSTM(source_vocab_size, hidden_size, batch_size).to(device)
decoder1 = DecoderLSTM(hidden_size, target_vocab_size+1, batch_size).to(device) # +1 is a wordaround for ignore_index field of NLLLoss
trainIters(encoder1, decoder1, 133317, print_every=10)

(10 0%) 9.5885
[['We' 'polio' 'Dear' 'pedal' 'harness' 'analogies' 'workshop' 'talents'
  'occasional' '.' '</s>' '</s>' 'Kenyan' 'Kenyan']
 ['We' 'Brendan' 'takes' 'subject' 'equals' 'to' 'Genspace' '13th' '.'
  '</s>' '</s>' 'Kenyan' 'Kenyan' 'Kenyan']
 ['We' 'polio' 'symptom' 'catadores' 'increasingly' 'cookies' 'effort'
  '.' 'rested' 'JN' '.' '.' '</s>' '</s>']
 ['We' 'polio' 'Left' 'eureka' 'municipal' 'sings' 'to' 'to' 'effort'
  'mounting' 'exploding' 'midair' 'Stanley' '</s>']
 ['We' 'We' 'can' 'the' 'the' '.' '.' 'the' 'effort' '</s>' '</s>' '</s>'
  'Kenyan' 'Kenyan']
 ['We' 'explores' 'can' '</s>' '</s>' '</s>' '</s>' 'Kenyan' 'Kenyan'
  'Kenyan' 'Kenyan' 'Kenyan' 'Kenyan' 'Kenyan']
 ['We' 'job' 'the' 'initiatives' 'store' 'can' 'the' 'yearly' '</s>'
  '</s>' 'Kenyan' 'Kenyan' 'Kenyan' 'Kenyan']
 ['We' 'job' 'the' 'the' 'LOL' 'LOL' 'wandering' '</s>' 'compute'
  'adolescence' 'TiVo' '</s>' '</s>' 'Kenyan']
 ['We' '.' 'plea' 'nonetheless' '.' '</s>' '</s>' 'Kenyan' 'Kenyan'


(110 0%) 2.4073
[['We' 'do' 'it' 'it' 'it' 'the' 'the' 'the' 'of' '.' '</s>' '.' '.' '.']
 ['We' 'it' 'it' 'to' 'to' 'to' 'to' '.' '.' '</s>' '.' '.' '.' '.']
 ['We' 'do' 'all' 'all' 'of' 'the' 'of' 'of' 'this' 'this' '.' '.' '</s>'
  '</s>']
 ['We' 'do' 'all' 'of' 'this' 'to' 'to' 'the' 'of' 'of' 'this' '.' '.'
  '</s>']
 ['We' 'you' 'can' 'can' 'the' 'of' 'of' 'the' 'of' '.' '</s>' '.' '.'
  '.']
 ['We' 'you' 'can' '.' '.' '</s>' '</s>' '.' '.' '.' '.' '.' '.' 'the']
 ['We' 'can' 'can' 'you' 'you' 'can' 'the' '.' '</s>' '.' '.' '.' '.' '.']
 ['We' 'can' 'can' 'the' 'of' 'of' 'the' '.' '.' '.' '.' '</s>' '</s>'
  '.']
 ['We' 'you' 'it' '.' '.' '</s>' '.' '.' '.' '.' '.' '.' 'the' 'the']
 ['We' 'you' ',' 'we' 'the' 'a' 'to' 'to' 'to' 'to' 'this' '.' '</s>' '.']]
(120 0%) 1.7500
[['We' 'do' 'it' 'up' 'and' 'the' 'the' 'the' 'of' '.' '</s>' '</s>' '.'
  '.']
 ['We' 'it' 'it' 'to' 'to' 'do' 'to' '.' '.' '</s>' '.' '.' '.' '.']
 ['We' 'do' 'all' 'all' 'of' 'the' 'world' 'looking' 'this' 't

(210 0%) 0.3749
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  '</s>' '.' '.']
 ['We' 'it' 'takes' 'weeks' 'to' 'perform' 'our' '<unk>' '.' '</s>'
  '</s>' '.' '.' '.']
 ['We' 'do' 'fly' 'all' 'over' 'the' 'world' 'looking' 'for' 'this'
  'thing' '.' '</s>' '</s>']
 ['We' 'do' 'all' 'of' 'this' 'to' 'understand' 'the' 'chemistry' 'of'
  'one' 'molecule' '.' '</s>']
 ['We' 'you' 'can' 'imagine' 'the' 'scale' 'of' 'the' 'effort' '.' '</s>'
  '.' '.' '.']
 ['We' 'you' 'very' 'much' '.' '</s>' '</s>' '.' '.' '.' '.' '.' '.' '.']
 ['We' 'can' 'mimic' 'what' 'you' 'can' 'see' '.' '</s>' '.' '.' '.' '.'
  '.']
 ['We' 'can' 'program' 'the' 'hundreds' 'of' 'muscles' 'in' 'your' 'arm'
  '.' '</s>' '</s>' '.']
 ['We' 'was' 'terribly' 'dangerous' '.' '</s>' '</s>' '.' '.' '.' '.' '.'
  '.' '.']
 ['We' 'now' ',' 'we' 'have' 'a' 'real' 'technology' 'to' 'do' 'this' '.'
  '</s>' '</s>']]
(220 0%) 0.3563
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  '</s>

(300 0%) 0.2742
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  '</s>' '.' '.']
 ['We' 'it' 'takes' 'weeks' 'to' 'perform' 'our' '<unk>' '.' '</s>'
  '</s>' '.' '.' '.']
 ['We' 'also' 'fly' 'all' 'over' 'the' 'world' 'looking' 'for' 'this'
  'thing' '.' '</s>' '</s>']
 ['We' 'do' 'all' 'of' 'this' 'to' 'understand' 'the' 'chemistry' 'of'
  'one' 'molecule' '.' '</s>']
 ['We' 'you' 'can' 'imagine' 'the' 'scale' 'of' 'the' 'effort' '.' '</s>'
  '</s>' '.' '.']
 ['We' 'you' 'very' 'much' '.' '</s>' '</s>' '.' '.' '.' '.' '.' '.' '.']
 ['We' 'can' 'mimic' 'what' 'you' 'can' 'see' '.' '</s>' '.' '.' '.' '.'
  '.']
 ['We' 'can' 'program' 'the' 'hundreds' 'of' 'muscles' 'in' 'your' 'arm'
  '.' '</s>' '</s>' '.']
 ['We' 'was' 'terribly' 'dangerous' '.' '</s>' '</s>' '.' '.' '.' '.' '.'
  '.' '.']
 ['We' 'now' ',' 'we' 'have' 'a' 'real' 'technology' 'to' 'do' 'this' '.'
  '</s>' '</s>']]
(310 0%) 0.2674
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  

(390 0%) 0.2125
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  '</s>' '.' '.']
 ['We' 'it' 'takes' 'weeks' 'to' 'perform' 'our' '<unk>' '.' '</s>'
  '</s>' '.' '.' '.']
 ['We' 'also' 'fly' 'all' 'over' 'the' 'world' 'looking' 'for' 'this'
  'thing' '.' '</s>' '</s>']
 ['We' 'do' 'all' 'of' 'this' 'to' 'understand' 'the' 'chemistry' 'of'
  'one' 'molecule' '.' '</s>']
 ['We' 'you' 'can' 'imagine' 'the' 'scale' 'of' 'the' 'effort' '.' '</s>'
  '</s>' '.' '.']
 ['You' 'you' 'very' 'much' '.' '</s>' '</s>' '.' '.' '.' '.' '.' '.' '.']
 ['You' 'can' 'mimic' 'what' 'you' 'can' 'see' '.' '</s>' '.' '.' '.' '.'
  '.']
 ['You' 'can' 'program' 'the' 'hundreds' 'of' 'muscles' 'in' 'your' 'arm'
  '.' '</s>' '</s>' '.']
 ['You' 'was' 'terribly' 'dangerous' '.' '</s>' '</s>' '.' '.' '.' '.'
  '.' '.' '.']
 ['We' 'now' ',' 'we' 'have' 'a' 'real' 'technology' 'to' 'do' 'this' '.'
  '</s>' '</s>']]
(400 0%) 0.2049
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>

(480 0%) 0.1417
[['We' 'blow' 'it' 'up' 'and' 'look' 'at' 'the' 'pieces' '.' '</s>'
  '</s>' '.' '.']
 ['And' 'it' 'takes' 'weeks' 'to' 'perform' 'our' '<unk>' '.' '</s>'
  '</s>' '.' '.' '.']
 ['We' 'also' 'fly' 'all' 'over' 'the' 'world' 'looking' 'for' 'this'
  'thing' '.' '</s>' '</s>']
 ['We' 'do' 'all' 'of' 'this' 'to' 'understand' 'the' 'chemistry' 'of'
  'one' 'molecule' '.' '</s>']
 ['So' 'you' 'can' 'imagine' 'the' 'scale' 'of' 'the' 'effort' '.' '</s>'
  '</s>' '.' '.']
 ['Thank' 'you' 'very' 'much' '.' '</s>' '</s>' '.' '.' '.' '.' '.' '.'
  '.']
 ['You' 'can' 'mimic' 'what' 'you' 'can' 'see' '.' '</s>' '.' '.' '.' '.'
  '.']
 ['You' 'can' 'program' 'the' 'hundreds' 'of' 'muscles' 'in' 'your' 'arm'
  '.' '</s>' '</s>' '.']
 ['It' 'was' 'terribly' 'dangerous' '.' '</s>' '</s>' '.' '.' '.' '.' '.'
  '.' '.']
 ['But' 'now' ',' 'we' 'have' 'a' 'real' 'technology' 'to' 'do' 'this'
  '.' '</s>' '</s>']]


KeyboardInterrupt: 